### Install packages

In [1]:
! pip install -r requirements.txt --quiet

### Load .env file (Copy .env-sample to .env and update accordingly)

Set the appropriate environment variables below:



In [2]:
from dotenv import load_dotenv
import os

load_dotenv(override=True) # take environment variables from .env.


azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_openai_deployment_name = os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]
azure_openai_key = os.environ["AZURE_OPENAI_API_KEY"]
azure_openai_api_version = os.environ["AZURE_OPENAI_API_VERSION"]


## Write output to a CSV file

In [ ]:
import csv
def write_markdown_table_to_csv(markdown_content, output_file):
    lines = [line.strip() for line in markdown_content.strip().split('\n') if line.strip()]
    # Filter out separator lines (those that start with |---)
    data_lines = [line for line in lines if not set(line.replace('|', '').strip()) <= set('-:')]
    rows = [ [cell.strip() for cell in line.strip('|').split('|')] for line in data_lines ]
    with open(output_file, "w", encoding="utf-8", newline="") as f:
        writer = csv.writer(f)
        writer.writerows(rows)

## Use Azure OpenAI to analyze sentiment & pain points in a survey

In [ ]:

from openai import AzureOpenAI
from openai.types.chat import ChatCompletion, ChatCompletionSystemMessageParam, ChatCompletionUserMessageParam, ChatCompletionMessage, ChatCompletionMessageParam
from azure.identity.aio import DefaultAzureCredential, get_bearer_token_provider
import pandas as pd


token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

client = AzureOpenAI(
    api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    api_key=azure_openai_key,
    azure_ad_token_provider=token_provider if not azure_openai_key else None
)

def process_csv_files(folder_path):
    """
    Process all CSV files in the specified folder.
    Args:
        folder_path (str): The path to the folder containing CSV files.
        output_folder (str): The path to the folder where output CSVs will be saved.
 
    """
    assistant_system_message = """
You are an AI assistant that analyzes sentiment data. Each row in the dataset contains a question in the 'Question:Name' column and a corresponding value in the 'Response Value' column. 

Below are the only sentiment analysis questions:

Q1. "Please provide additional detail to help us better understand the issue:" → Category: General  
Q2. "Please tell us specifically why you provided the above answer regarding the service you received from" → Category: General  
Q3. "Additional comments for RSA online support tools?" → Category: Tools  
Q4. "What was the primary source of your dissatisfaction?" → Category: TSE  
Q5. "Other, please specify:" → Category: TSE  
Q6. "Please tell us specifically why you provided the above answer regarding the RSA Product." → Category: Product  

For the above sentiment analysis questions : The "Response Value" column may contain useful feedback, irrelevant text, or no meaningful content. Your task is to intelligently identify and extract only actual pain points—statements that reflect dissatisfaction, frustration, pending task or problems experienced by the user.

Ignore responses that are:
- Blank or contain placeholder text (e.g., "N/A", "None", "No comments")
- Positive or neutral feedback
- Unrelated or unclear

"""
    query = """
1. Summarize the overall sentiment in percentage like how much percentage is positive and how much percentage is negative. And then list the summary of most positive comments from all the questions in the survey.

2. Create a table with the following columns:
- Sentiment Analysis Question
- Category
- Tools Pain Points
- TSE Pain Points
- Product Pain Points
- General Pain Points

Instructions:
1. For each row in the data, include only the sentiment analysis questions listed in the system prompt, even if the same question appears multiple times in the dataset.
2. Determine the Category using the mappings provided in the system prompt.
3. For each occurrence of a sentiment analysis question, analyze the value in the "Response Value" column and determine if it contains a valid pain point.
4. If a pain point is present, summarize it concisely and place it strictly in the appropriate column based on the category:
   - If the category is Tools, fill in the Tools Pain Points column.
   - If TSE, fill in the TSE Pain Points column.
   - If Product, fill in the Product Pain Points column.
   - If General, fill in the General Pain Points column.
   Please do not copy the entire response value; instead, summarize the pain point in a few words or a short sentence.
5. Do not place pain points in multiple columns or in a column that does not match the question's category. Ensure not to mix categories.
6. If multiple pain points are found for the same question across different rows, include each one in the output table.
7. Leave the pain point columns blank if the value in the "Response Value" column does not contain a valid pain point.
"""

    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            file_path = os.path.join(folder_path, filename)
            print(f"Processing file: {file_path}")

            df = pd.read_csv(file_path)
            csv_content = df.to_csv(index=False)

            query_message = f"{query}\nRelevant data: {csv_content}"
            messages =   [
            ChatCompletionSystemMessageParam(role="system", content=assistant_system_message),
            ChatCompletionUserMessageParam(role="user", content=query_message),
            ] 
        
            response: ChatCompletion = client.chat.completions.create(
            model=azure_openai_deployment_name,
                messages=messages,
                max_tokens=800,
                temperature=0.7,
                top_p=0.95,
                frequency_penalty=0,
                presence_penalty=0,
                stop=None,
                stream=False
            )

            message: ChatCompletionMessage = response.choices[0].message
            print(f"{message.content}")


            output_folder= r"YOur path to the output folder"
            # Try to parse the output as CSV and save it
            output_file = os.path.join(output_folder, f"output_{filename}")
            write_markdown_table_to_csv(message.content, output_file)

            print(f"Saved output to {output_file}")


process_csv_files(r"Your path to the input folder")